In [2]:
%run nearest_mrt_station_to_bus_stops.ipynb

['https://ltafarecard.s3.ap-southeast-1.amazonaws.com/202409/origin_destination_bus_202409.zip?X-Amz-Security-Token=IQoJb3JpZ2luX2VjEP7%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEaDmFwLXNvdXRoZWFzdC0xIkcwRQIhAOrFHmo9GbM%2B387oC98gLva6LLZ5O50056xti6SiMllgAiBM3MnZazBXnZG81o%2BzJFajHNVIDSGOLHwwPL0B3R%2BZsirCBQh3EAQaDDM0MDY0NTM4MTMwNCIMU3DeWPzjdPo6SIlsKp8FCo7j%2FlX2uJYqIWjYMvDHfddWsDM1fqFFs6wMh6Itf7pJD9WQbkU6kMmdxXzyZ8w7%2BF6nwDlgcZDQQMVhok4%2BOcnrJTKM9k5T9jpjPlcn8hQAmJ5UantYHKSwl3HJTUis07U%2FTmn8Lj%2F7CXor5QvKHZvoYpZLYNnUW1nAbr%2FA3DZA3%2FvJnWi2F4ZIEynEt5tf2u6YhZb8AX8paLMDTDhl%2Fle6dgXmMCOBd%2FXjoOsn%2FU3xyqSNEazO%2F2orWg8sUjDJ9OMBMKgrL%2FcBuLo0yQAWWtruMCv8mcJjmOG3cmDEQ7kq6fF1r1TbvxFHMi8F0peo%2Br41MNj1prHpO%2B2vPgtUrq%2FMqW2o8CIBegpdnV1dl8vQOsUCfuC6uhLGIF%2Bryjq5oOXNngsxZaBB96POidkhFAoEcBKt9YawwNVlAsvxLuKjWJ0%2BYmgLax3%2F13FREyllRezfu3o676PQe%2BiEXU3wve1Mj1b0k6VR6jKMzVcyYDtCVBDVZTR1ow9e3irHdWKAl8GHMUQX%2FJMR6%2F14tr36rRSsI%2BJ8UwXgnE95mHCpR1NS9u7mliPcC%2BUsZD5EUb0Uio2LyBdylIO4pPnrkN8VfUzpP6lSz6nzhaGz

25548it [00:21, 1183.32it/s]


# MRT Stations Data

In [3]:
# mrt stations geodataframe
import os
os.environ['OGR_GEOMETRY_ACCEPT_UNCLOSED_RING'] = 'NO'

mrt_stations_gdf = geopandas.read_file('./TrainStation_Jul2024/RapidTransitSystemStation.shp')
mrt_stations_gdf_3857 = mrt_stations_gdf.to_crs(3857)
mrt_stations_gdf_3857

,TYP_CD,STN_NAM,ATTACHEMEN,TYP_CD_DES,STN_NAM_DE,geometry
0,0,None,None,MRT,GALI BATU DEPOT,"POLYGON ((11549880.181 155286.262, 11549893.09..."
1,0,None,None,MRT,HILLVIEW MRT STATION,"POLYGON ((11551320.421 151685.479, 11551324.86..."
2,0,None,None,MRT,BEAUTY WORLD MRT STATION,"POLYGON ((11552265.127 149269.279, 11552255.26..."
3,0,None,None,MRT,HUME MRT STATION,"POLYGON ((11551478.151 150855.083, 11551485.53..."
4,0,None,None,MRT,BUKIT PANJANG MRT STATION,"POLYGON ((11550666.118 153603.709, 11550698.62..."
...,...,...,...,...,...,...
225,0,None,None,MRT,KATONG PARK MRT STATION,"POLYGON ((11564432.509 144548.028, 11564447.75..."
226,0,None,None,MRT,MARINE TERRACE MRT STATION,"POLYGON ((11567908.523 145472.086, 11567908.42..."
227,0,None,None,MRT,TANJONG RHU MRT STATION,"POLYGON ((11563110.672 144334.178, 11563108.03..."
228,0,None,None,MRT,MARINE PARADE MRT STATION,"POLYGON ((11566600.443 144955.407, 11566582.69..."


In [4]:
mrt_stations_gdf2 = geopandas.read_file('./mrt_stations/mrt_stations.shp')

mrt_stations_gdf2["lat_lng"] = geopandas.points_from_xy(mrt_stations_gdf2["LATITUDE"], mrt_stations_gdf2["LONGITUDE"])
mrt_stations_gdf2["line"] = mrt_stations_gdf2["CODE"].str.slice(0, 2) 

mrt_lines_code = ["NS", "EW", "CC", "NE", "TE", "DT"]

mrt_gdf_wo_lrt = mrt_stations_gdf2[mrt_stations_gdf2["line"].isin(mrt_lines_code)].reset_index()
mrt_gdf_wo_lrt = mrt_gdf_wo_lrt.drop("index", axis = 1)

mrt_gdf_wo_lrt["STN_NAM_DE"] = mrt_gdf_wo_lrt["BUILDING"].str.split('(').str[0].str.strip()
mrt_stations_3857_2 = mrt_gdf_wo_lrt.merge(mrt_stations_gdf_3857, how = "left", on = "STN_NAM_DE")

mrt_stations_3857_2

,CODE,STATION_NA,LINE,COLOR,OPENING,TYPE,SEARCHVAL,BLK_NO,ROAD_NAME,BUILDING,...,LONGITUDE,geometry_x,lat_lng,line,STN_NAM_DE,TYP_CD,STN_NAM,ATTACHEMEN,TYP_CD_DES,geometry_y
0,NS1,Jurong East,North-South Line,Red,1990-03-10,MRT,JURONG EAST MRT STATION (EW24 / NS1),10,JURONG EAST STREET 12,JURONG EAST MRT STATION (EW24 / NS1),...,103.742286332403,POINT (17869.057 35038.969),POINT (1.333 103.742),NS,JURONG EAST MRT STATION,0,None,None,MRT,"POLYGON ((11548513.722 148484.605, 11548515.37..."
1,NS2,Bukit Batok,North-South Line,Red,1990-03-10,MRT,BUKIT BATOK MRT STATION (NS2),10,BUKIT BATOK CENTRAL,BUKIT BATOK MRT STATION (NS2),...,103.749566478309,POINT (18679.322 36794.926),POINT (1.349 103.75),NS,BUKIT BATOK MRT STATION,0,None,None,MRT,"POLYGON ((11549318.598 150099.924, 11549295.94..."
2,NS3,Bukit Gombak,North-South Line,Red,1990-03-10,MRT,BUKIT GOMBAK MRT STATION (NS3),802,BUKIT BATOK WEST AVENUE 5,BUKIT GOMBAK MRT STATION (NS3),...,103.751790910733,POINT (18926.913 37854.036),POINT (1.359 103.752),NS,BUKIT GOMBAK MRT STATION,0,None,None,MRT,"POLYGON ((11549629.35 151190.998, 11549628.967..."
3,NS4,Choa Chu Kang,North-South Line,Red,1990-03-10,MRT,CHOA CHU KANG MRT STATION (NS4),10,CHOA CHU KANG AVENUE 4,CHOA CHU KANG MRT STATION (NS4),...,103.744370779756,POINT (18101.247 40812.122),POINT (1.385 103.744),NS,CHOA CHU KANG MRT STATION,0,None,None,MRT,"POLYGON ((11548794.664 154207.851, 11548779.42..."
4,NS5,Yew Tee,North-South Line,Red,1996-02-10,MRT,YEW TEE MRT STATION (NS5),61,CHOA CHU KANG DRIVE,YEW TEE MRT STATION (NS5),...,103.747405150236,POINT (18438.983 42158.018),POINT (1.398 103.747),NS,YEW TEE MRT STATION,0,None,None,MRT,"POLYGON ((11549092.655 155476.939, 11549088.92..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206,TE20,Marina Bay,Thomson-East Coast Line,Brown,2022-11-13,MRT,MARINA BAY MRT STATION (TE20),25,PARK STREET,MARINA BAY MRT STATION (TE20),...,103.855309167928,POINT (30447.379 28579.995),POINT (1.275 103.855),TE,MARINA BAY MRT STATION,0,None,CE2_MRB STN.zip,MRT,"POLYGON ((11561070.528 141987.904, 11561054.72..."
207,TE20,Marina Bay,Thomson-East Coast Line,Brown,2022-11-13,MRT,MARINA BAY MRT STATION (TE20),25,PARK STREET,MARINA BAY MRT STATION (TE20),...,103.855309167928,POINT (30447.379 28579.995),POINT (1.275 103.855),TE,MARINA BAY MRT STATION,0,None,None,MRT,"POLYGON ((11561039.437 141991.719, 11561036.58..."
208,TE20,Marina Bay,Thomson-East Coast Line,Brown,2022-11-13,MRT,MARINA BAY MRT STATION (TE20),25,PARK STREET,MARINA BAY MRT STATION (TE20),...,103.855309167928,POINT (30447.379 28579.995),POINT (1.275 103.855),TE,MARINA BAY MRT STATION,0,None,None,MRT,"POLYGON ((11561017.163 142126.143, 11561004.19..."
209,TE20,Marina Bay,Thomson-East Coast Line,Brown,2022-11-13,MRT,MARINA BAY MRT STATION (TE20),25,PARK STREET,MARINA BAY MRT STATION (TE20),...,103.855309167928,POINT (30447.379 28579.995),POINT (1.275 103.855),TE,MARINA BAY MRT STATION,0,None,None,MRT,"POLYGON ((11561104.449 141953.568, 11561129.46..."


# Brown Line

In [5]:
brown_line = ['WOODLANDS NORTH MRT STATION', 'WOODLANDS MRT STATION', 'WOODLANDS SOUTH MRT STATION',
              'SPRINGLEAF MRT STATION', 'LENTOR MRT STATION', 'MAYFLOWER MRT STATION',
              'BRIGHT HILL MRT STATION', 'UPPER THOMSON MRT STATION', 'CALDECOTT MRT STATION',
              'STEVENS MRT STATION', 'NAPIER MRT STATION', 'ORCHARD BOULEVARD MRT STATION',
              'ORCHARD MRT STATION', 'GREAT WORLD MRT STATION', 'HAVELOCK MRT STATION',
              'OUTRAM PARK MRT STATION', 'MAXWELL MRT STATION', 'SHENTON WAY MRT STATION',
              'MARINA BAY MRT STATION', 'GARDENS BY THE BAY MRT STATION', 'TANJONG RHU MRT STATION',
              'KATONG PARK MRT STATION', 'TANJONG KATONG MRT STATION', 'MARINE PARADE MRT STATION',
              'MARINE TERRACE MRT STATION', 'SIGLAP MRT STATION', 'BAYSHORE MRT STATION',
              'BEDOK SOUTH MRT STATION', 'SUNGEI BEDOK MRT STATION']

# filter brown line stations from mrt stations geodataframe
brown_line_stations_gdf = mrt_stations_gdf_3857[mrt_stations_gdf_3857['STN_NAM_DE'].isin(brown_line)]

# order brown line stations based on list
brown_line_stations_gdf['STN_NAM_DE'] = pd.Categorical(brown_line_stations_gdf['STN_NAM_DE'],
                                                       categories = brown_line,
                                                       ordered = True)
brown_line_stations_gdf = brown_line_stations_gdf.sort_values('STN_NAM_DE')
brown_line_stations_gdf


,TYP_CD,STN_NAM,ATTACHEMEN,TYP_CD_DES,STN_NAM_DE,geometry
192,0,None,None,MRT,WOODLANDS NORTH MRT STATION,"POLYGON ((11553290.504 161340.499, 11553308.78..."
196,0,None,None,MRT,WOODLANDS MRT STATION,"POLYGON ((11553655.571 159844.445, 11553714.07..."
177,0,None,None,MRT,WOODLANDS MRT STATION,"POLYGON ((11553305.733 159892.909, 11553302.44..."
191,0,None,None,MRT,WOODLANDS SOUTH MRT STATION,"POLYGON ((11554340.403 158819.168, 11554344.10..."
187,0,None,None,MRT,SPRINGLEAF MRT STATION,"POLYGON ((11556983.209 155700.488, 11556972.59..."
110,0,None,None,MRT,LENTOR MRT STATION,"POLYGON ((11559127.808 154037.405, 11559127.68..."
219,0,None,None,MRT,MAYFLOWER MRT STATION,"POLYGON ((11559099.098 152920.462, 11559094.58..."
186,0,None,None,MRT,BRIGHT HILL MRT STATION,"POLYGON ((11558731.827 151811.306, 11558686.82..."
217,0,None,None,MRT,UPPER THOMSON MRT STATION,"POLYGON ((11558480.214 150916.092, 11558483.67..."
218,0,None,None,MRT,CALDECOTT MRT STATION,"POLYGON ((11559462.068 148993.215, 11559463.61..."


In [7]:
# ordered list of brown line station coordinates
brown_line_stations_gdf['coordinates'] = brown_line_stations_gdf['geometry'].centroid
brown_line_stations_gdf

# create a linestring for the brown line route
from shapely.geometry import LineString
brown_line_route = LineString(brown_line_stations_gdf['coordinates'])

In [11]:
brown_line_stations_gdf

,TYP_CD,STN_NAM,ATTACHEMEN,TYP_CD_DES,STN_NAM_DE,geometry,coordinates
192,0,None,None,MRT,WOODLANDS NORTH MRT STATION,"POLYGON ((11553290.504 161340.499, 11553308.78...",POINT (11553368.462 161246.769)
196,0,None,None,MRT,WOODLANDS MRT STATION,"POLYGON ((11553655.571 159844.445, 11553714.07...",POINT (11553618.123 159879.837)
177,0,None,None,MRT,WOODLANDS MRT STATION,"POLYGON ((11553305.733 159892.909, 11553302.44...",POINT (11553422.897 159964.08)
191,0,None,None,MRT,WOODLANDS SOUTH MRT STATION,"POLYGON ((11554340.403 158819.168, 11554344.10...",POINT (11554283.645 158879.97)
187,0,None,None,MRT,SPRINGLEAF MRT STATION,"POLYGON ((11556983.209 155700.488, 11556972.59...",POINT (11556958.488 155652.161)
110,0,None,None,MRT,LENTOR MRT STATION,"POLYGON ((11559127.808 154037.405, 11559127.68...",POINT (11559009.325 154184.852)
219,0,None,None,MRT,MAYFLOWER MRT STATION,"POLYGON ((11559099.098 152920.462, 11559094.58...",POINT (11559058.674 152756.428)
186,0,None,None,MRT,BRIGHT HILL MRT STATION,"POLYGON ((11558731.827 151811.306, 11558686.82...",POINT (11558629.643 151772.539)
217,0,None,None,MRT,UPPER THOMSON MRT STATION,"POLYGON ((11558480.214 150916.092, 11558483.67...",POINT (11558628.2 150785.724)
218,0,None,None,MRT,CALDECOTT MRT STATION,"POLYGON ((11559462.068 148993.215, 11559463.61...",POINT (11559431.34 148882.994)


### Brown Line Route Visualisation

In [25]:
import folium
import geopandas
from shapely.geometry import mapping

# convert brown_line_route from LineString to geodataframe
brown_line_route_gdf = geopandas.GeoDataFrame(geometry = [brown_line_route], crs = "EPSG:3857")
brown_line_route_gdf_4326 = brown_line_route_gdf.to_crs(4326)

# create base map
brown_line_map = folium.Map(location = (1.359394, 103.814301), zoom_start = 12)

# add brown line route layer
folium.GeoJson(
    data = mapping(brown_line_route_gdf_4326),
    name = 'TEL',
    style_function = lambda x: {
        'color': 'brown',
        'weight': 4
    }
).add_to(brown_line_map)

brown_line_map



In [27]:
from shapely.geometry import LineString

# combine bus routes
bus_routes_combined = bus_routes_gdf_3857.groupby(['ServiceNo', 'Direction']).apply(lambda x: LineString(x.geometry.tolist())).reset_index()
bus_routes_combined.columns = ['ServiceNo', 'Direction', 'geometry']
bus_routes_combined

/var/folders/d0/twkl0gkd7lv67r2q_sxfmpnw0000gn/T/ipykernel_60509/1680479668.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  bus_routes_combined = bus_routes_gdf_3857.groupby(['ServiceNo', 'Direction']).apply(lambda x: LineString(x.geometry.tolist())).reset_index()


,ServiceNo,Direction,geometry
0,10,1,LINESTRING (11570925.355747303 150749.03122670...
1,10,2,LINESTRING (11551610.561061133 144087.88490860...
2,100,1,LINESTRING (11562943.630025776 150347.12028417...
3,100,2,LINESTRING (11553774.516701924 145960.08849871...
4,100A,1,LINESTRING (11562943.630025776 150347.12028417...
...,...,...,...
719,992,1,LINESTRING (11549515.381030776 150294.50067879...
720,992,2,LINESTRING (11547835.02338783 151709.373424986...
721,993,1,LINESTRING (11548357.891036088 148329.89223418...
722,9A,1,LINESTRING (11569327.609406518 147460.11332459...


In [79]:
# calculate the overlapping distance of each bus route and the brown line
buffer_distance = 150
buffered_brown_line = brown_line_route.buffer(buffer_distance)

overlap_distance_output = []

for i, route in tqdm(bus_routes_combined.iterrows()):
    overlap = buffered_brown_line.intersection(route.geometry)
    if overlap.is_empty:
        overlap_distance_output.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': 0,
            'MRT line': 'Thomson-East Coast Line',
            'Intersection': overlap
            })
    else:
        overlap_distance_output.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': overlap.length,
            'MRT line': 'Thomson-East Coast Line',
            'Intersection': overlap
            })

overlap_distance = geopandas.GeoDataFrame(overlap_distance_output)
overlap_distance

724it [00:00, 2861.53it/s]


,Bus Service,Direction,Overlap Distance,MRT line,Intersection
0,10,1,303.158745,Thomson-East Coast Line,LINESTRING (11560562.07903124 142416.229493252...
1,10,2,304.681655,Thomson-East Coast Line,LINESTRING (11560279.846380834 142213.23057919...
2,100,1,303.158745,Thomson-East Coast Line,LINESTRING (11560562.07903124 142416.229493252...
3,100,2,305.338292,Thomson-East Coast Line,LINESTRING (11560279.846380834 142213.23057919...
4,100A,1,0.000000,Thomson-East Coast Line,LINESTRING Z EMPTY
...,...,...,...,...,...
719,992,1,0.000000,Thomson-East Coast Line,LINESTRING Z EMPTY
720,992,2,0.000000,Thomson-East Coast Line,LINESTRING Z EMPTY
721,993,1,0.000000,Thomson-East Coast Line,LINESTRING Z EMPTY
722,9A,1,0.000000,Thomson-East Coast Line,LINESTRING Z EMPTY


In [80]:
# filter bus routes with overlap
bus_routes_overlap = overlap_distance[overlap_distance['Overlap Distance'] > 0].sort_values(by = 'Overlap Distance', ascending = False)
bus_routes_overlap

,Bus Service,Direction,Overlap Distance,MRT line,Intersection
349,36,1,13497.671113,Thomson-East Coast Line,MULTILINESTRING ((11570480.682728108 145953.86...
351,36B,1,11775.414400,Thomson-East Coast Line,MULTILINESTRING ((11569767.053458788 145922.54...
392,47,1,6834.465678,Thomson-East Coast Line,MULTILINESTRING ((11569869.7078534 146101.9133...
225,196e,1,6707.001743,Thomson-East Coast Line,MULTILINESTRING ((11569767.053458788 145922.54...
350,36A,1,6150.759588,Thomson-East Coast Line,MULTILINESTRING ((11570480.682728108 145953.86...
...,...,...,...,...,...
182,178,1,125.862879,Thomson-East Coast Line,LINESTRING (11553397.54702328 159976.868225107...
183,178,2,124.004316,Thomson-East Coast Line,LINESTRING (11553273.544894883 159976.13166109...
619,911A,1,104.939940,Thomson-East Coast Line,LINESTRING (11553375.39444461 159966.066837988...
552,851e,1,46.125602,Thomson-East Coast Line,LINESTRING (11559562.696989702 142667.75756188...


### Overlap Visualisation (example - will be extended for the rest of the routes)

In [117]:
# create base map
brown_line_overlap_map = folium.Map(location = (1.359394, 103.814301), zoom_start = 12)

# add brown line layer
folium.GeoJson(
    data = mapping(brown_line_route_gdf_4326),
    name = 'TEL',
    style_function = lambda x: {
        'color': 'brown',
        'weight': 3
    }
).add_to(brown_line_overlap_map)

# add buffered brown line layer
buffered_brown_line_gdf = geopandas.GeoDataFrame(geometry = [buffered_brown_line], crs = "EPSG:3857")
buffered_brown_line_gdf_4326 = buffered_brown_line_gdf.to_crs(4326)

folium.GeoJson(
    data = mapping(buffered_brown_line_gdf_4326),
    name = 'Buffered TEL',
    style_function = lambda x: {
        'color': 'brown',
        'weight': 0.5
    }
).add_to(brown_line_overlap_map)

# add bus route layer
bus_36 = bus_routes_combined[(bus_routes_combined['ServiceNo'] == '36') & (bus_routes_combined['Direction'] == 1)]
bus_36_gdf = geopandas.GeoDataFrame(bus_36, geometry = bus_36['geometry'], crs = "EPSG:3857")
bus_36_gdf_4326 = bus_36_gdf.to_crs(4326)

folium.GeoJson(
    data = mapping(bus_36_gdf_4326),
    name = 'Bus 36',
    style_function = lambda x: {
        'weight': 4
    }
).add_to(brown_line_overlap_map)

# add intersection layer
bus_36_overlap = [line for line in bus_routes_overlap.head(1)['Intersection'].values[0].geoms]
bus_36_overlap_gdf = geopandas.GeoDataFrame(geometry = bus_36_overlap, crs = "EPSG:3857")
bus_36_overlap_gdf_4326 = bus_36_overlap_gdf.to_crs(4326)

folium.GeoJson(
    bus_36_overlap_gdf_4326,
    name = 'Overlap',
    style_function = lambda x: {
        'color': 'red'
    }
).add_to(brown_line_overlap_map)

# add toggle for layers
folium.LayerControl(position = 'bottomright').add_to(brown_line_overlap_map)

brown_line_overlap_map
# brown_line_overlap_map.save('(directory)/brown_line_overlap_map.html')

In [110]:
import os

# Get the current working directory
current_directory = os.getcwd()
print("Current Working Directory:", current_directory)

Current Working Directory: /Users/ko-shyan/Desktop/dsa4264/project/dsa4264_geospatial/workingfiles


# Blue Line

In [ ]:
blue_line = [
    'SUNGEI KADUT MRT STATION', 'BUKIT PANJANG MRT STATION', 'CASHEW MRT STATION', 'HILLVIEW MRT STATION', 'HUME MRT STATION', 'BEAUTY WORLD MRT STATION', 'KING ALBERT PARK MRT STATION', 
    'SIXTH AVENUE MRT STATION', 'TAN KAH KEE MRT STATION', 'BOTANIC GARDENS MRT STATION', 'STEVENS MRT STATION', 'NEWTON MRT STATION', 
    'LITTLE INDIA MRT STATION', 'ROCHOR MRT STATION', 'BUGIS MRT STATION', 'PROMENADE MRT STATION', 'BAYFRONT MRT STATION', 'DOWNTOWN MRT STATION', 'TELOK AYER MRT STATION', 
    'CHINATOWN MRT STATION', 'FORT CANNING MRT STATION', 'BENCOOLEN MRT STATION', 'JALAN BESAR MRT STATION', 'BENDEMEER MRT STATION', 
    'GEYLANG BAHRU MRT STATION', 'MATTAR MRT STATION', 'MACPHERSON MRT STATION', 'UBI MRT STATION', 'KAKI BUKIT MRT STATION', 'BEDOK NORTH MRT STATION', 
    'BEDOK RESERVOIR MRT STATION', 'TAMPINES WEST MRT STATION', 'TAMPINES MRT STATION', 'TAMPINES EAST MRT STATION', 'UPPER CHANGI MRT STATION', 'EXPO MRT STATION'
]

# filter brown line stations from mrt stations geodataframe
blue_line_stations_gdf = mrt_stations_gdf_3857[mrt_stations_gdf_3857['STN_NAM_DE'].isin(blue_line)]

# order brown line stations based on list
blue_line_stations_gdf['STN_NAM_DE'] = pd.Categorical(blue_line_stations_gdf['STN_NAM_DE'],
                                                       categories = blue_line,
                                                       ordered = True)
blue_line_stations_gdf = blue_line_stations_gdf.sort_values('STN_NAM_DE')
blue_line_stations_gdf



In [12]:
# ordered list of blue line station coordinates
blue_line_stations_gdf['coordinates'] = blue_line_stations_gdf['geometry'].centroid
blue_line_stations_gdf

# create a linestring for the blue line route
from shapely.geometry import LineString
blue_line_route = LineString(blue_line_stations_gdf['coordinates'])

In [ ]:
from shapely.geometry import LineString

# combine bus routes
bus_routes_combined = bus_routes_gdf_3857.groupby(['ServiceNo', 'Direction']).apply(lambda x: LineString(x.geometry.tolist())).reset_index()
bus_routes_combined.columns = ['ServiceNo', 'Direction', 'geometry']
bus_routes_combined

In [ ]:
# calculate the overlapping distance of each bus route and the blue line
buffer_distance = 150
buffered_blue_line = blue_line_route.buffer(buffer_distance)

overlap_distance_output = []

for i, route in tqdm(bus_routes_combined.iterrows()):
    overlap = buffered_blue_line.intersection(route.geometry)
    if overlap.is_empty:
        overlap_distance_output.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': 0,
            'MRT line': 'Downtown Line'
            })
    else:
        overlap_distance_output.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': overlap.length,
            'MRT line': 'Downtown Line'
            })

overlap_distance_blue = pd.DataFrame(overlap_distance_output)
overlap_distance_blue

In [ ]:
# filter bus routes with overlap with blue line
bus_routes_overlap_blue = overlap_distance_blue[overlap_distance_blue['Overlap Distance'] > 0].sort_values(by = 'Overlap Distance', ascending = False)
bus_routes_overlap_blue

In [16]:
bus_routes_overlap = pd.concat([bus_routes_overlap, bus_routes_overlap_blue], axis = 0)

# Purple Line

In [ ]:
purple_line = [
    'HARBOURFRONT MRT STATION',
    'OUTRAM PARK MRT STATION',
    'CHINATOWN MRT STATION',
    'CLARKE QUAY MRT STATION',
    'DHOBY GHAUT MRT STATION',
    'LITTLE INDIA MRT STATION',
    'FARRER PARK MRT STATION',
    'BOON KENG MRT STATION',
    'POTONG PASIR MRT STATION',
    'WOODLEIGH MRT STATION',
    'SERANGOON MRT STATION',
    'KOVAN MRT STATION',
    'HOUGANG MRT STATION',
    'BUANGKOK MRT STATION',
    'SENGKANG MRT STATION',
    'PUNGGOL MRT STATION'
]


# filter purple line stations from mrt stations geodataframe
purple_line_stations_gdf = mrt_stations_gdf_3857[mrt_stations_gdf_3857['STN_NAM_DE'].isin(purple_line)]

# order purple line stations based on list
purple_line_stations_gdf['STN_NAM_DE'] = pd.Categorical(purple_line_stations_gdf['STN_NAM_DE'],
                                                       categories = purple_line,
                                                       ordered = True)
purple_line_stations_gdf = purple_line_stations_gdf.sort_values('STN_NAM_DE')
purple_line_stations_gdf

In [18]:
# ordered list of purple line station coordinates
purple_line_stations_gdf['coordinates'] = purple_line_stations_gdf['geometry'].centroid
purple_line_stations_gdf

# create a linestring for the purple line route
from shapely.geometry import LineString
purple_line_route = LineString(purple_line_stations_gdf['coordinates'])

In [ ]:
from shapely.geometry import LineString

# combine bus routes
bus_routes_combined = bus_routes_gdf_3857.groupby(['ServiceNo', 'Direction']).apply(lambda x: LineString(x.geometry.tolist())).reset_index()
bus_routes_combined.columns = ['ServiceNo', 'Direction', 'geometry']
bus_routes_combined

In [ ]:
# calculate the overlapping distance of each bus route and the purple line
buffer_distance = 150
buffered_purple_line = purple_line_route.buffer(buffer_distance)

overlap_distance_output = []

for i, route in tqdm(bus_routes_combined.iterrows()):
    overlap = buffered_purple_line.intersection(route.geometry)
    if overlap.is_empty:
        overlap_distance_output.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': 0,
            'MRT line': 'North-East Line'
            })
    else:
        overlap_distance_output.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': overlap.length,
            'MRT line': 'North-East Line'
            })

overlap_distance_purple = pd.DataFrame(overlap_distance_output)
overlap_distance_purple

In [ ]:
# filter bus routes with overlap with purple line
bus_routes_overlap_purple = overlap_distance_purple[overlap_distance_purple['Overlap Distance'] > 0].sort_values(by = 'Overlap Distance', ascending = False)
bus_routes_overlap_purple

In [22]:
bus_routes_overlap = pd.concat([bus_routes_overlap, bus_routes_overlap_purple], axis = 0)

# Red Line

In [ ]:
red_line = mrt_stations_3857_2[mrt_stations_3857_2["line"] == "NS"]["STATION_NA"].tolist()

# filter brown line stations from mrt stations geodataframe
red_line_stations_gdf = mrt_stations_3857_2[mrt_stations_3857_2['line'] == "NS"]

red_line_stations_gdf["order"] = pd.to_numeric(red_line_stations_gdf["CODE"].str.slice(2, None))

# order brown line stations based on list
red_line_stations_gdf = red_line_stations_gdf.sort_values('order')
red_line_stations_gdf

In [ ]:
# ordered list of brown line station coordinates
red_line_stations_gdf['coordinates'] = red_line_stations_gdf['geometry_y'].centroid
red_line_stations_gdf


# create a linestring for the brown line route
from shapely.geometry import LineString
red_line_route = LineString(red_line_stations_gdf['coordinates'])
red_line_route

In [ ]:
buffered_red_line = red_line_route.buffer(buffer_distance)

overlap_distance_output_red = []

for i, route in tqdm(bus_routes_combined.iterrows()):
    overlap = buffered_red_line.intersection(route.geometry)
    if overlap.is_empty:
        overlap_distance_output_red.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': 0,
            'MRT line': 'North-South Line'
            })
    else:
        overlap_distance_output_red.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': overlap.length,
            'MRT line': 'North-South Line'
            })

overlap_distance_red = pd.DataFrame(overlap_distance_output_red)
overlap_distance_red

In [ ]:
bus_routes_overlap_red = overlap_distance_red[overlap_distance_red['Overlap Distance'] > 0].sort_values(by = 'Overlap Distance', ascending = False)
bus_routes_overlap_red

In [27]:
bus_routes_overlap = pd.concat([bus_routes_overlap, bus_routes_overlap_red], axis = 0)

In [ ]:
bus_routes_overlap

# Green line

In [ ]:
green_line = mrt_stations_3857_2[mrt_stations_3857_2["line"] == "EW"]["STATION_NA"].tolist()
green_line_stations_gdf = mrt_stations_3857_2[mrt_stations_3857_2['line'] == "EW"]

green_line_stations_gdf["order"] = pd.to_numeric(green_line_stations_gdf["CODE"].str.slice(2, None))

# order brown line stations based on list
green_line_stations_gdf = green_line_stations_gdf.sort_values('order')
green_line_stations_gdf

In [ ]:
green_line_stations_gdf = geopandas.GeoDataFrame(green_line_stations_gdf, geometry='geometry_y')

green_line_stations_gdf['coordinates'] = green_line_stations_gdf['geometry_y'].centroid
green_line_stations_gdf

# create a linestring for the green line route
from shapely.geometry import LineString
green_line_route = LineString(green_line_stations_gdf['coordinates'])
green_line_route

In [ ]:
buffered_green_line = green_line_route.buffer(buffer_distance)

overlap_distance_output_green = []

for i, route in tqdm(bus_routes_combined.iterrows()):
    overlap = buffered_green_line.intersection(route.geometry)
    if overlap.is_empty:
        overlap_distance_output_green.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': 0,
            'MRT line': 'East-West Line'
            })
    else:
        overlap_distance_output_green.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': overlap.length,
            'MRT line': 'East-West Line'
            })

overlap_distance_green = pd.DataFrame(overlap_distance_output_green)
overlap_distance_green

In [ ]:
bus_routes_overlap_green = overlap_distance_green[overlap_distance_green['Overlap Distance'] > 0].sort_values(by = 'Overlap Distance', ascending = False)
bus_routes_overlap_green

# Circle Line

In [ ]:
circle_line = mrt_stations_3857_2[mrt_stations_3857_2["line"] == "CC"]["STATION_NA"].tolist()

In [ ]:
circle_line_stations_gdf = mrt_stations_3857_2[mrt_stations_3857_2['line'] == "CC"]

circle_line_stations_gdf["order"] = pd.to_numeric(green_line_stations_gdf["CODE"].str.slice(2, None))

# order  stations based on list
circle_line_stations_gdf = circle_line_stations_gdf.sort_values('order')
circle_line_stations_gdf

In [ ]:
circle_line_stations_gdf = geopandas.GeoDataFrame(circle_line_stations_gdf, geometry='geometry_y')

circle_line_stations_gdf['coordinates'] = circle_line_stations_gdf['geometry_y'].centroid
circle_line_stations_gdf

# create a linestring for the green line route
from shapely.geometry import LineString
circle_line_route = LineString(circle_line_stations_gdf['coordinates'])
circle_line_route

In [ ]:
buffered_circle_line = circle_line_route.buffer(buffer_distance)

overlap_distance_output_circle = []

for i, route in tqdm(bus_routes_combined.iterrows()):
    overlap = buffered_circle_line.intersection(route.geometry)
    if overlap.is_empty:
        overlap_distance_output_circle.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': 0,
            'MRT line': 'Circle Line'
            })
    else:
        overlap_distance_output_circle.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': overlap.length,
            'MRT line': 'Circle Line'
            })

overlap_distance_circle = pd.DataFrame(overlap_distance_output_circle)
overlap_distance_circle

In [ ]:
bus_routes_overlap_circle = overlap_distance_circle[overlap_distance_circle['Overlap Distance'] > 0].sort_values(by = 'Overlap Distance', ascending = False)
bus_routes_overlap_circle